---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
"""
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);
""";

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [2]:
def answer_one():
        
    movie_choice_df = pd.read_csv('Employee_Movie_Choices.txt', sep = '\t')
    movie_choice_df.rename(columns = {'#Employee': 'Employee'}, inplace = True)
    
    bipartite_graph = nx.from_pandas_dataframe(movie_choice_df, 'Employee', 'Movie')
    
    return bipartite_graph 

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [3]:
def answer_two():
    
    graph = answer_one()

    # Create a dictionary that maps the 'type' attribute values to the entire set of nodes of the graph. 
    node_attribute_type_employees = dict.fromkeys(employees, 'employee')
    node_attribute_type_movies = dict.fromkeys(movies, 'movie')

    node_attribute_type = node_attribute_type_employees
    node_attribute_type.update(node_attribute_type_movies)

    # Set node attributes all named 'type' for all nodes in the graph. 
    nx.set_node_attributes(graph, 'type', node_attribute_type)

    return graph

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [4]:
def answer_three():
        
    graph = answer_two()
    weighted_projected_graph = bipartite.weighted_projected_graph(graph, employees)
    
    return weighted_projected_graph

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [5]:
def answer_four():
        
    # First, add all possible edges between all employees to graph that aren't already in graph. Give these edges attribute of 
    # {'weight': 0}. [I.e. if there are n possible edges between m nodes, and there are n-k unique edges, add the remaining 
    # k unique edges, and give all of them the attribute of {'weight': 0}]. 

    # Since there are 8 employees, i.e. 8 nodes, the number of all possible edges are 8*7/2 = 28.  

    graph = answer_three()
    missing_edges_list = list(nx.non_edges(graph))
    graph.add_edges_from(missing_edges_list, weight = 0)

    # Second, import 'Employee_Relationships.txt' to a dataframe. Then create a second graph that takes each row of this dataframe
    # as an edge, and the set of unique values of both columns 'n1' and 'n2' as the set of nodes of this graph. 

    relationship_df = pd.read_csv('Employee_Relationships.txt', sep = '\t', header = None,
                                  names = ['n1', 'n2', 'relationship score'])
    graph2 = nx.from_pandas_dataframe(relationship_df, 'n1', 'n2', edge_attr='relationship score')

    # Third, combine the two graphs (graph, graph2) together into one graph, then create a dataframe that takes each edge
    # of this combined graph as the row with two columns for the two attributes without specifying the nodes of each edge. 

    combined_graph = nx.compose(graph, graph2)

    list_of_edges = combined_graph.edges(data=True)
    attribute_values = [(x[2]['relationship score'], x[2]['weight']) for x in list_of_edges]

    attributes_df = pd.DataFrame(attribute_values, columns = ['relationship score', 'weight'])
    
    return attributes_df.corr(method='pearson').iloc[0][1]